In [2]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel

In [3]:
# We create a spark session and use the spark context within the spark session because we want both dataframes and rdds
# 
ss = SparkSession.builder.appName("DataProc").config("spark.executor.cores", "4").config("spark.driver.memory", "4g").config("spark.executor.memory", "4g").config("spark.executor.memoryOverhead", "4g").getOrCreate()
sc = ss.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/08 16:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# we load up an rdd from the paths text file
paths_rdd = sc.textFile("data/wet.paths") 

In [5]:
paths_rdd.count()

100000

In [8]:
# we take approximately sample_size paths from the samples rdd. path contains like 30k crawls so we don't want huge samples
sample_size = 1
sampled_paths = sc.parallelize(paths_rdd.takeSample(withReplacement=False, num=sample_size, seed=0))
# the above example isn't exactly the most optimal sampling technique, but it allows us to be exact with sample size
# also the strings aren't very big, and there isn't a ton of data, so this doesn't really cost us much time

In [9]:
sampled_paths.count()

1

In [10]:
# this is just something we need to prepend to every path so we can create it into a link to download
url_head = "https://data.commoncrawl.org/" 

In [11]:
# now we have an rdd of just urls 
urls_rdd = sampled_paths.map(lambda x: url_head + x) 

In [12]:
urls_rdd.take(1)

['https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-47/segments/1762439344200.50/wet/CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz']

In [13]:
# we use requests to get a http response from the common crawl server
def get_request(url): 
    import requests
    import random
    import time
    
    jitter = random.random()
    time.sleep(jitter)
    response = requests.get(url, stream=True) 
    if response.status_code == 200: 
        return response 
    else:
        # Here we implement an exponential back off strategy to make sure we're not overloading the server
        # Here we wait for a total of (2^11)/10 = 204.8 seconds or about 3.5 minutes max to get a response from the server
        # So total wait time can be a bit over 5 minutes
        for i in range(12):
            jitter = random.random()
            time.sleep((2**i + jitter)/10)
            response = requests.get(url, stream=True)
            if response.status_code == 200: 
                # Request was successful 
                return response 
        return None 

In [14]:
# This is to have our responses RDD be filtered so we don't have any Nonetypes
responses_rdd = urls_rdd.map(get_request).persist(StorageLevel.MEMORY_AND_DISK).filter(lambda x: x is not None)

In [15]:
# we convert the responses into gzip files in memory 
def get_zipped(response): 
    import io 
    return io.BytesIO(response.content) 

In [16]:
zipped_files = responses_rdd.map(get_zipped).persist(StorageLevel.MEMORY_AND_DISK) 
responses_rdd.unpersist()

PythonRDD[16] at RDD at PythonRDD.scala:56

In [17]:
zipped_files.take(1) 

In [18]:
# we unzip the zipped files also within memory 
def unzip(zipped): 
    import gzip 
    with gzip.GzipFile(fileobj=zipped) as decompressed_file: 
        return decompressed_file.read()

In [19]:
unzipped_rdd = zipped_files.map(unzip).persist(StorageLevel.MEMORY_AND_DISK) 
zipped_files.unpersist()

PythonRDD[17] at RDD at PythonRDD.scala:56

In [20]:
type(unzipped_rdd.take(1))

list

In [21]:
# we convert the unzipped bytes into usable strings 
str_rdd = unzipped_rdd.map(lambda x: x.decode("utf-8"))
unzipped_rdd.unpersist()
str_rdd.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[25] at RDD at PythonRDD.scala:56

In [22]:
str_rdd.count()

1

In [23]:

#

In [24]:
#str_rdd.is_cached
str_rdd.unpersist()

PythonRDD[25] at RDD at PythonRDD.scala:56

In [25]:
# as you can see this is a plaintext snippet of the plaintext WET file. we need to process this string rdd to do our analysis. 
# we need to strip away the description data and access just the text. ideally we get rid of line breaks and stuff like that too. 
# then throw it into language detection, lda, and so on 
print(str_rdd.take(1)[0][:10000]) 

WARC/1.0
WARC-Type: warcinfo
WARC-Date: 2025-11-21T04:54:24Z
WARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz
WARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>
Content-Type: application/warc-fields
Content-Length: 372

Software-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243
Extracted-Date: Fri, 21 Nov 2025 04:54:24 GMT
robots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
isPartOf: CC-MAIN-2025-47
operator: Common Crawl Admin (info@commoncrawl.org)
description: Wide crawl of the web for November 2025
publisher: Common Crawl



WARC/1.0
WARC-Type: conversion
WARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB
WARC-Date: 2025-11-15T21:55:56Z
WARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>
WARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>
WARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2GJXBHDFZWZ
WARC-Identified-Content-Language: zho
Content-Type: text/plain


In [26]:
# Here we look at the raw string and see where to split it 
str_rdd.take(1)[0][:10000]

'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2025-11-21T04:54:24Z\r\nWARC-Filename: CC-MAIN-20251115201147-20251115231147-00569.warc.wet.gz\r\nWARC-Record-ID: <urn:uuid:27966d2c-0e0c-4bd3-ab9c-2e67f0e6f7eb>\r\nContent-Type: application/warc-fields\r\nContent-Length: 372\r\n\r\nSoftware-Info: ia-web-commons.3.0.1-SNAPSHOT-20251103031243\r\nExtracted-Date: Fri, 21 Nov 2025 04:54:24 GMT\r\nrobots: checked via crawler-commons 1.6-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)\r\nisPartOf: CC-MAIN-2025-47\r\noperator: Common Crawl Admin (info@commoncrawl.org)\r\ndescription: Wide crawl of the web for November 2025\r\npublisher: Common Crawl\r\n\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB\r\nWARC-Date: 2025-11-15T21:55:56Z\r\nWARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>\r\nWARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>\r\nWARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2G

In [27]:
# Since there is a \n\r\n between every double line break, we split it accordingly 
# We also get rid of the first 3 chunks as it essentially is just WET file metadata that exists per file; we're only interested in per-site data
split_rdd = str_rdd.flatMap(lambda x: x.split("\n\r\n")[3:])
split_rdd.persist(StorageLevel.MEMORY_AND_DISK)
split_rdd.getStorageLevel()

StorageLevel(True, True, False, False, 1)

In [28]:
list_of_chunks = split_rdd.take(12)

In [29]:
list_of_chunks[0] 

'WARC/1.0\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://022hezi.com/tag/%E4%B9%9D%E5%8D%81%E5%85%AB\r\nWARC-Date: 2025-11-15T21:55:56Z\r\nWARC-Record-ID: <urn:uuid:02be2c7b-d20b-4e22-8a93-401bf6614637>\r\nWARC-Refers-To: <urn:uuid:a192054e-7280-436a-a2f9-919f3c2558e3>\r\nWARC-Block-Digest: sha1:NZ7VKS26DB7EWWQRGSA3B2GJXBHDFZWZ\r\nWARC-Identified-Content-Language: zho\r\nContent-Type: text/plain\r\nContent-Length: 1931\r'

In [30]:
print((split_rdd.count())//2) 

22135


In [31]:
type_rdd = str_rdd.map(lambda x: str(type(x))) 

In [32]:
type_rdd.take(1)

["<class 'str'>"]

In [33]:
split_rdd = str_rdd.map(lambda x: x[0].split("\n\r\n")) 

In [34]:
listed = str_rdd.take(1)[0].split("\n\r\n") 

In [35]:
ss.stop()